# Imports

In [ ]:
import sys

sys.path.append("..")

from solarv2 import *

In [ ]:
init_notebook_mode(all_interactive=True)

# Parameters

In [ ]:
params.simulate_dead_area = False

# Save options
params.save_figures = True

# Plotting options
params.individual_plots = np.arange(1, 10, 1)
params.show_figures = True
params.label_font_size = 16
params.tick_font_size = 16
params.title_font_size = 18

# Transform for real readout
params.detector_x = params.quadrant_size * 8
params.detector_y = params.quadrant_size * 8

# Filters for post processing if not using filter parameters file
params.min_score = -1.0
params.max_score = 1.0
params.min_track_length = 0
params.max_track_length = np.inf
params.max_tracks = 1
params.max_light = np.inf
params.min_light = 0
params.max_z = np.inf

In [ ]:
metrics_file = "0977563451/metrics_0977563451.pkl"
if params.lifetime > 0:
    metrics_file = metrics_file.replace(".pkl", f"_lt{params.lifetime:.3}.pkl")

# To load all filter parameters from saved JSON if it is not None and exists. Will overwrite all variables.
filter_file = None  # "combined/filter_parameters_27786.json"

In [ ]:
params.output_folder = "_".join(metrics_file.split(".")[0].split("/")[:-1])
if params.output_folder == "":
    params.output_folder = "combined"
if params.simulate_dead_area:
    params.work_path = "DA"

In [ ]:
recal_params()

# File loading

In [ ]:
# Load metrics from pickle file
if not os.path.isfile(metrics_file):
    metrics = combine_metrics()
else:
    with open(metrics_file, "rb") as f:
        metrics = pickle.load(f)

In [ ]:
cached_metrics = metrics
if filter_file is not None and os.path.isfile(filter_file):
    with open(filter_file, "r") as f:
        filter_settings = json.load(f)
        metrics = filter_metrics(
            metrics,
            **filter_settings,
        )
        globals().update(filter_settings)
else:
    metrics = filter_metrics(metrics)

In [ ]:
# Save parameters to JSON just in case
params_to_json(
    f"{params.output_folder}/analysis_parameters_{params.output_folder}.json"
)

# Analysis

In [ ]:
metrics

## dQ/dx

### Statistical plots

In [ ]:
plot_track_stats(
    metrics,
    limit_xrange=True,
    empty_ratio_lims=(0.0, 1),
    lognorm=False,
    min_entries=2,
    min_score=0.5,
    bins=[40, 40],
    profile=False,
)
if params.show_figures:
    plt.show()
else:
    plt.close("all")

In [ ]:
plot_track_stats(
    metrics,
    limit_xrange=True,
    empty_ratio_lims=(0.0, 1),
    lognorm=True,
    min_entries=2,
    min_score=0.5,
    bins=[40, 40],
    profile=True,
)
plt.close("all")

### Individual plots

In [ ]:
for event_idx in tqdm(params.individual_plots, leave=False):
    if event_idx in metrics:
        for track_idx, values in metrics[event_idx].items():
            if not isinstance(track_idx, str) and track_idx > 0:
                dQ_array = values["dQ"]
                dh = values["dx"]
                plot_dQ(dQ_array, event_idx, track_idx, dh, interpolate=False)

                if params.show_figures:
                    plt.show()
                else:
                    plt.close("all")

# Other

## Track angles

In [ ]:
# Cos^2 of track angles
def plot_track_angles(metrics):
    cos_x = []
    cos_y = []
    cos_z = []
    vectors = []
    for idx, metric in metrics.items():
        for track_idx, track in metric.items():
            if type(track) is dict:
                if "RANSAC_score" in track and track["RANSAC_score"] < 0.5:
                    continue
                if "Fit_norm" in track and track["Fit_norm"] < 1:
                    continue
                if "Fit_line" in track:
                    cos_x.append(
                        track["Fit_line"].direction.cosine_similarity([1, 0, 0])
                    )
                    cos_y.append(
                        track["Fit_line"].direction.cosine_similarity([0, 1, 0])
                    )
                    cos_z.append(
                        track["Fit_line"].direction.cosine_similarity([0, 0, 1])
                    )
                    vectors.append(track["Fit_line"].direction.to_array())

    vectors = np.array(vectors)
    cos_x = np.array(cos_x)
    cos_y = np.array(cos_y)
    cos_z = np.array(cos_z)

    fig, ax = plt.subplots(2, 3, figsize=(18, 12))

    ax[0, 0].hist(vectors[:, 0], bins=20)
    ax[0, 0].set_xlabel("X vector component")
    ax[0, 1].hist(vectors[:, 1], bins=20)
    ax[0, 1].set_xlabel("Y vector component")
    ax[0, 2].hist(vectors[:, 2], bins=20)
    ax[0, 2].set_xlabel("Z vector component")

    ax[1, 0].hist(abs(cos_x), bins=20)
    ax[1, 0].set_xlabel("Cosine similarity to x-axis")
    ax[1, 1].hist(abs(cos_y), bins=20)
    ax[1, 1].set_xlabel("Cosine similarity to y-axis")
    ax[1, 2].hist(abs(cos_z), bins=20)
    ax[1, 2].set_xlabel("Cosine similarity to z-axis")

    return fig, ax

In [ ]:
plot_track_angles(metrics)
plt.show()

## Heat map

In [ ]:
def sectorize_dqdx(metrics, bin_size=(32, 32)):
    temp_df = get_track_stats(metrics)
    temp_df = temp_df[temp_df["track_score"] > 0.5]
    dQdx_df = temp_df.explode("track_dQdx")
    dQdx_df["position"] = temp_df["track_points"].explode()
    dQdx_df["x"] = dQdx_df["position"].apply(lambda x: x[0])
    dQdx_df["y"] = dQdx_df["position"].apply(lambda x: x[1])
    dQdx_df["z"] = dQdx_df["position"].apply(lambda x: x[2])
    dQdx_df = dQdx_df[(dQdx_df["track_dQdx"] > 0)].drop("position", axis=1)

    # Define the bin edges for a range of coordinates
    x_bins = np.arange(
        -params.detector_x / 2 - bin_size[0],
        params.detector_x / 2 + 2 * bin_size[0],
        bin_size[0],
    )
    y_bins = np.arange(
        -params.detector_y / 2 - bin_size[1],
        params.detector_y / 2 + 2 * bin_size[1],
        bin_size[1],
    )

    # Cut the data into bins
    dQdx_df["x_bin"] = pd.cut(dQdx_df["x"], bins=x_bins, labels=False)
    dQdx_df["y_bin"] = pd.cut(dQdx_df["y"], bins=y_bins, labels=False)
    last_x = dQdx_df["x_bin"].max()
    last_y = dQdx_df["y_bin"].max()
    dQdx_df["x_bin"] = dQdx_df["x_bin"].apply(
        lambda x: 1 if x == 0 else last_x - 1 if x == last_x else x
    )
    dQdx_df["y_bin"] = dQdx_df["y_bin"].apply(
        lambda x: 1 if x == 0 else last_y - 1 if x == last_y else x
    )

    # Create a DataFrame for all possible bin combinations
    all_bins = pd.DataFrame(
        [(x, y) for x in dQdx_df["x_bin"].unique() for y in dQdx_df["y_bin"].unique()],
        columns=["x_bin", "y_bin"],
    )

    # Merge the actual data with the placeholder DataFrame
    dQdx_df = pd.merge(
        all_bins, dQdx_df.reset_index(), on=["x_bin", "y_bin"], how="left"
    ).fillna(0)

    # Combine the bins into a single sector identifier
    dQdx_df["sector"] = dQdx_df.apply(lambda row: (row["x_bin"], row["y_bin"]), axis=1)

    return dQdx_df

In [ ]:
dQdx_df = sectorize_dqdx(metrics, bin_size=(32, 32))

In [ ]:
plt.scatter(dQdx_df["x"], dQdx_df["y"], c=dQdx_df["track_dQdx"], s=0.1)
plt.xticks(np.arange(-params.detector_x / 2, params.detector_x / 2 + 32, 32))
plt.yticks(np.arange(-params.detector_y / 2, params.detector_y / 2 + 32, 32))
plt.grid()
plt.colorbar()

In [ ]:
counts = dQdx_df.pivot_table(
    index="y_bin", columns="x_bin", values="track_dQdx", fill_value=0, aggfunc="count"
)
counts_cut = dQdx_df[
    (1500 < dQdx_df["track_dQdx"]) & (dQdx_df["track_dQdx"] < 3000)
].pivot_table(
    index="y_bin", columns="x_bin", values="track_dQdx", fill_value=0, aggfunc="count"
)

plt.pcolormesh(counts_cut / counts)
cbar = plt.colorbar(label="Ratio per sector")
plt.gca().set_aspect("equal", adjustable="box")
plt.xticks(counts.columns)
plt.yticks(counts.index)
plt.xlabel("X bin")
plt.ylabel("Y bin")
plt.title("Ratio of dQdx counts per sector")
plt.show()

In [ ]:
t = dQdx_df.groupby("event")["track_dQdx"].mean()
t[((t < 3000) & (1500 < t))].index

In [ ]:
fig, ax = plt.subplots(
    dQdx_df["y_bin"].nunique(), dQdx_df["x_bin"].nunique(), figsize=(20, 20)
)
for sector in dQdx_df["sector"].unique():
    x = int(sector[0]) - 1
    y = 4 - int(sector[1]) + 1
    dQdx_df[dQdx_df["sector"] == sector]["track_dQdx"].hist(
        ax=ax[y, x], bins=np.arange(0, 12e3, 400)
    )